In [1]:
import os 
# Now run the prebaked systems example
os.chdir('c:\\Users\\Danie\\OneDrive\\Documents\\Coding\\pysystemtrade\\pysystemtrade\\examples\\introduction')

In [2]:
import sys
import os

# Add the parent directory of pysystemtrade to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))

In [ ]:
import logging

# Set logging level to INFO to suppress DEBUG messages
logging.getLogger().setLevel(logging.INFO)

# Import systemtest
from systems.systemtest import systemtest
from sysdata.sim.csv_futures_sim_data import csvFuturesSimData

# Initialize the data
data = csvFuturesSimData()

# Create and run the system with JPY instrument
my_system = systemtest(instruments=["SOFR", "ES", "GC"])
print(my_system.portfolio.get_notional_position("JPY").tail(5))


In [ ]:
import logging

# Set logging level to INFO to suppress DEBUG messages
logging.getLogger().setLevel(logging.INFO)

# Import systemtest
from systems.systemtest import systemtest
from sysdata.sim.csv_futures_sim_data import csvFuturesSimData

# Initialize the data
data = csvFuturesSimData()

system = systemtest(instruments=["SOFR", "JPY", "MXD"])
results = analyze_strategies(instruments=["SOFR", "JPY", "MXD"])

In [ ]:
import logging

# Set logging level to INFO to suppress DEBUG messages
logging.getLogger().setLevel(logging.INFO)

# First, import the correct system
from systems.provided.example.simplesystem import simplesystem
from sysdata.sim.csv_futures_sim_data import csvFuturesSimData
from sysdata.config.configdata import Config

# Initialize the data
data = csvFuturesSimData()

# Create config
my_config = Config("systems.provided.example.simplesystemconfig.yaml")

# Create and run the system with both config and data
my_system = simplesystem(config=my_config, data=data)
# print(my_system.portfolio.get_notional_position("SOFR").tail(5))
print(my_system.portfolio.get_notional_position("JPY").tail(5))

In [ ]:
import logging

# Set logging level to INFO to suppress DEBUG messages
logging.getLogger().setLevel(logging.INFO)

# First, import the correct system
from systems.provided.example.simplesystem import simplesystem
from sysdata.sim.csv_futures_sim_data import csvFuturesSimData
from sysdata.config.configdata import Config

# Initialize the data
data = csvFuturesSimData()

# Create config
my_config = Config("systems.provided.example.simplesystemconfig.yaml")

# Create and run the system with both config and data
my_system = simplesystem(config=my_config, data=data)
print(my_system.portfolio.get_notional_position("SOFR").tail(5))

In [ ]:
# Import matplotlib
import matplotlib.pyplot as plt

# Plot the equity curve
my_system.accounts.portfolio().curve().plot()
plt.title('Test System Performance')
plt.ylabel('Portfolio Value')
plt.xlabel('Time')
plt.show()

In [ ]:
# Get account statistics for the chapter 15 system
my_system.accounts.portfolio().percent.stats()